In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

In [2]:
text4000 = pd.read_csv(".//Data/4000/4000-stories-VAD.csv")

In [3]:
text4000.head(1)

,url,length,title,text_no,author,story,valence,arousal,dominance,text_id,...,290,291,292,293,294,295,296,297,298,299
0,https://americanliterature.com/author/eleanor-...,15044,"Peace on Earth, Good-Will to Dogs",0,Eleanor Hallowell Abbott,"PART I\n\nIf you don't like Christmas stories,...",0.592896,0.397839,0.569567,0_eleanor-hallowell-abbott-peace-on-earth-good...,...,3.793141,3.837345,3.778354,-0.815515,-0.72044,-10.738245,-8.765683,0.875089,-10.176691,1.736791


In [4]:
text4000.keys()

Index(['url', 'length', 'title', 'text_no', 'author', 'story', 'valence',
       'arousal', 'dominance', 'text_id',
       ...
       '290', '291', '292', '293', '294', '295', '296', '297', '298', '299'],
      dtype='object', length=317)

In [5]:
texts= list(text4000.story.values)

In [6]:
# text_limiter=1000000 #10000 zeichen
joined_text = " ".join(texts)
# limited_text = joined_text[:text_limiter]

In [7]:
# Tokenizer erstellen und Texte darauf anwenden
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(joined_text.lower())

In [8]:
context_words = 10
input_words = []
next_words = []
words_limiter=100000 #limitiert die anzahl an Trainingsdaten
for i in range(len(tokens) - context_words):
    input_words.append(tokens[i:i + context_words])
    next_words.append(tokens[i + context_words])
    if(len(next_words)>=words_limiter):break
len(next_words)

100000

In [9]:
abc=100
print(input_words[abc])
print(next_words[abc])

['keyboard', 'and', 'very', 'large', 'sized', 'dogs', 'shall', 'romp', 'through', 'every']
page


In [10]:
input_tokens = [token for sublist in input_words for token in sublist]
unique_tokens = np.unique(np.concatenate((input_tokens, next_words))) #alle Token 108282 || in 50000 Token sind 7359 unique
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [11]:
len(unique_tokens)

9803

In [12]:
X = np.zeros((len(input_words), context_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [13]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
    Y[i, unique_token_index[next_words[i]]] = 1

In [14]:
# Aufteilung der Trainings- und Validierungsdaten
split_index = int(len(X) * 0.95)
x_train, x_val = X[:split_index], X[split_index:]
y_train, y_val = Y[:split_index], Y[split_index:]


In [15]:
with open('FormatedData/100000/x_train.pickle', 'wb') as file:
    pickle.dump(x_train, file)
with open('FormatedData/100000/x_val.pickle', 'wb') as file:
    pickle.dump(x_val, file)
with open('FormatedData/100000/y_train.pickle', 'wb') as file:
    pickle.dump(y_train, file)
with open('FormatedData/100000/y_val.pickle', 'wb') as file:
    pickle.dump(y_val, file)

In [16]:
with open('FormatedData/100000/MetaData/context_words.pickle', 'wb') as file:
    pickle.dump(context_words, file)
with open('FormatedData/100000/MetaData/unique_tokens.pickle', 'wb') as file:
    pickle.dump(unique_tokens, file)
with open('FormatedData/100000/MetaData/unique_token_index.pickle', 'wb') as file:
    pickle.dump(unique_token_index, file)   